# Importing Libraries

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import contractions
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [2]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Reading the file and split into lines

In [4]:
# Read the csv file.

df2 = pd.read_csv("backend/data/raw/english_to_bangla.csv", encoding='utf-8')
df2.head()


,en,bn
0,a child in a pink dress is climbing up a set o...,একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্...
1,a girl going into a wooden building .,একটি মেয়ে শিশু একটি কাঠের বাড়িতে ঢুকছে
2,a little girl climbing into a wooden playhouse .,একটি বাচ্চা তার কাঠের খেলাঘরে উঠছে ।
3,a little girl climbing the stairs to her playh...,ছোট মেয়েটি তার খেলার ঘরের সিড়ি বেয়ে উঠছে
4,a little girl in a pink dress going into a woo...,গোলাপি জামা পড়া ছোট একটি মেয়ে একটি কাঠের তৈরি...


In [5]:
df2["en"][0]

'a child in a pink dress is climbing up a set of stairs in an entry way .'

In [6]:
df2["bn"][0]

'একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্রবেশ পথের সিঁড়ি বেয়ে উঠছে।'

In [ ]:
df1 = pd.read_csv("backend/data/raw/eng_to_bng.csv", encoding='utf-8') 
df1.head()

,en,bn
0,Go.,যান।
1,Go.,যা।
2,Run!,পালাও!
3,Run!,পালান!
4,Who?,কে?


In [8]:
df = pd.concat([df1, df2], ignore_index=True)
df.head()

,en,bn
0,Go.,যান।
1,Go.,যা।
2,Run!,পালাও!
3,Run!,পালান!
4,Who?,কে?


In [10]:
df[:50]

,en,bn
0,Go.,যান।
1,Go.,যা।
2,Run!,পালাও!
3,Run!,পালান!
4,Who?,কে?
5,Fire!,আগুন!
6,Help!,বাঁচাও!
7,Help!,বাঁচান!
8,Stop!,থামুন!
9,Stop!,থামো!


In [11]:
def contractions_func(v : str) -> str:
    return contractions.fix(v)

In [12]:
df["en"] = df["en"].apply(contractions_func)

In [13]:
df[:50]

,en,bn
0,Go.,যান।
1,Go.,যা।
2,Run!,পালাও!
3,Run!,পালান!
4,Who?,কে?
5,Fire!,আগুন!
6,Help!,বাঁচাও!
7,Help!,বাঁচান!
8,Stop!,থামুন!
9,Stop!,থামো!


In [14]:
df['en_word_length'] = df['en'].str.split().str.len()

In [15]:
df['bn_word_length'] = df['bn'].str.split().str.len()

In [16]:
print(f"Maximum length of english sentence : {max(df['en_word_length'])} \nMaximum length of bengali sentence : {max(df['bn_word_length'])}")

Maximum length of english sentence : 38 
Maximum length of bengali sentence : 32


In [17]:
df[:50]

,en,bn,en_word_length,bn_word_length
0,Go.,যান।,1,1
1,Go.,যা।,1,1
2,Run!,পালাও!,1,1
3,Run!,পালান!,1,1
4,Who?,কে?,1,1
5,Fire!,আগুন!,1,1
6,Help!,বাঁচাও!,1,1
7,Help!,বাঁচান!,1,1
8,Stop!,থামুন!,1,1
9,Stop!,থামো!,1,1


In [18]:
print(f"The number of sentences is : {len(df)}")

The number of sentences is : 43396


In [19]:
df = df[~((df["en_word_length"] > 26) | (df["bn_word_length"] > 26))]
df = df.reset_index(drop=True)

In [20]:
print(f"The number of sentences which has less than 27 is {len(df)}")

The number of sentences which has less than 27 is 43297


In [21]:
df.head()

,en,bn,en_word_length,bn_word_length
0,Go.,যান।,1,1
1,Go.,যা।,1,1
2,Run!,পালাও!,1,1
3,Run!,পালান!,1,1
4,Who?,কে?,1,1


In [22]:
type(df["en"][0])

str

In [23]:
type(df["bn"][0])

str

In [24]:
df["en"][0].lower()

'go.'

In [25]:
df["bn"][0].lower()

'যান।'

In [26]:
df.tail(5)

,en,bn,en_word_length,bn_word_length
43292,a man in a pink shirt climbs a rock face,গোলাপী শার্টের একটি লোক একটি শিলার উপরে উঠেছিল,10,8
43293,a man is rock climbing high in the air .,একজন মানুষ পাথরে চড়ছে অনেক উপরে,10,6
43294,a person in a red shirt climbing up a rock fac...,লাল শার্টের একজন ব্যক্তি সহায়তার হাতলগুলিতে ঢ...,16,12
43295,a rock climber in a red shirt .,লাল শার্টে একটি রক আরোহী,8,5
43296,a rock climber practices on a rock climbing wa...,একটি রক আরোহী একটি শৈল আরোহণ প্রাচীর এর উপর অন...,10,11


In [27]:
df["en"][43296]

'a rock climber practices on a rock climbing wall .'

In [28]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1



In [102]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# # Lowercase, trim, and remove non-letter characters
# def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
#     return s.strip()


In [29]:
# def unicodeToAscii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    s = s.lower().strip()
    s = s[:-1]
    #s = re.sub(r"([.!?])", r" \1", s)              # Add space before punctuation
    #s = re.sub(r"[^\w\s.!?]", r"", s)              # Remove non-word characters except punctuation
    return s.strip()


In [30]:
eng_text = df["en"][2]
print(f"The actual text is : {eng_text}")
processed_eng_text = normalizeString(eng_text)
print(f"The processed text is : {processed_eng_text}")

The actual text is : Run!
The processed text is : run


In [31]:
bng_text = df["bn"][2]
print(f"The actual text is : {bng_text}")
processed_bng_text = normalizeString(bng_text)
print(f"The processed text is : {processed_bng_text}")

The actual text is : পালাও!
The processed text is : পালাও


In [32]:
def readLangs(lang1, lang2, df):
    print("Reading lines...")

    pairs = []
    length = len(df)
    for i in range(length):
        pair = []
        bng_text = df[lang1][i]
        processed_bng_text = normalizeString(bng_text)
        pair.append(processed_bng_text)
        eng_text = df[lang2][i]
        processed_eng_text = normalizeString(eng_text)
        pair.append(processed_eng_text)
        pairs.append(pair)

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


# Prepareing data

In [33]:
MAX_LENGTH = 27

In [34]:
def prepareData(lang1, lang2, df):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, df)
    print("Read %s sentence pairs" % len(pairs))

    # pairs = filterPairs(pairs)
    # print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")

    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [35]:
input_lang, output_lang, pairs = prepareData('bn', 'en', df)

Reading lines...
Read 43297 sentence pairs
Counting words...
Counted words:
bn 18920
en 10320


In [36]:
print(pairs[:50])

[['যান', 'go'], ['যা', 'go'], ['পালাও', 'run'], ['পালান', 'run'], ['কে', 'who'], ['আগুন', 'fire'], ['বাঁচাও', 'help'], ['বাঁচান', 'help'], ['থামুন', 'stop'], ['থামো', 'stop'], ['থাম', 'stop'], ['নমস্কার', 'hello'], ['বুঝলাম', 'i see'], ['আমি চেষ্টা করি', 'i try'], ['একটু হাসুন', 'smile'], ['একটু হাসো', 'smile'], ['আক্রমণ', 'attack'], ['ওঠো', 'get up'], ['উঠুন', 'get up'], ['বুঝে গেছি', 'got it'], ['ধরেছি', 'got it'], ['বুঝেছো', 'got it'], ['বুঝেছেন', 'got it'], ['বুঝেছিস', 'got it'], ['আমি জানি', 'i know'], ['আমার জানা আছে', 'i know'], ['আমি হেরে গেছি', 'i lost'], ['আমার ১৯ বছর বয়স', 'i am 19'], ['আমি ঠিক আছি', 'i am ok'], ['শোন', 'listen'], ['শুনুন', 'listen'], ['কোন মতেই না', 'no way'], ['সত্যি', 'really'], ['তাই নাক', 'really'], ['ধন্যবাদ', 'thanks'], ['চেখে দেখুন', 'try it'], ['চেখে দেখো', 'try it'], ['আমরা জিতে গেছে', 'we won'], ['আমিই কেন', 'why me'], ['টমকে জিজ্ঞাসা করো', 'ask tom'], ['টমকে জিজ্ঞাসা করুন', 'ask tom'], ['টমকে জিজ্ঞাসা কর', 'ask tom'], ['আমাকে ডেক', 'call me'], [

In [37]:
print(f"Maximum length of english sentence : {max(df['en_word_length'])} \nMaximum length of bengali sentence : {max(df['bn_word_length'])}")

Maximum length of english sentence : 26 
Maximum length of bengali sentence : 26


# Encoder block

In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_vocab_size, hidden_size, dropout_p=0.1):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(input_vocab_size, hidden_size)
#         self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
#         self.dropout = nn.Dropout(dropout_p)

#     def forward(self, input_vector):
#         embedded = self.dropout(self.embedding(input_vector))
#         output, (hidden, cell) = self.lstm(embedded)
#         return output, (hidden, cell)


In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, hidden_size, dropout_p=0.1):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
#         self.dropout = nn.Dropout(dropout_p)

#     def forward(self, input):
#         embedded = self.dropout(self.embedding(input))
#         output, hidden = self.gru(embedded)
#         return output, hidden



# Attention Block

In [ ]:
# class BahdanauAttention(nn.Module):
#     def __init__(self, hidden_size):
#         super(BahdanauAttention, self).__init__()
#         self.Wa = nn.Linear(hidden_size, hidden_size)
#         self.Ua = nn.Linear(hidden_size * 2, hidden_size)  # 2x for bidirectional encoder
#         self.Va = nn.Linear(hidden_size, 1)

#     def forward(self, query, keys):
#         scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
#         scores = scores.squeeze(2).unsqueeze(1)
#         weights = F.softmax(scores, dim=-1)
#         context = torch.bmm(weights, keys)
#         return context, weights


# Decoder block with attention.

In [ ]:
# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_vocab_size, dropout_p=0.1):
#         super(AttnDecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(output_vocab_size, hidden_size)
#         self.attention = BahdanauAttention(hidden_size)
#         self.lstm = nn.LSTM(hidden_size * 3, hidden_size, batch_first=True)  # 1x embed + 2x context
#         self.out = nn.Linear(hidden_size, output_vocab_size)
#         self.dropout = nn.Dropout(dropout_p)

#         # Projection layers to reduce bi-LSTM encoder output to match decoder LSTM input size
#         self.reduce_h = nn.Linear(hidden_size * 2, hidden_size)
#         self.reduce_c = nn.Linear(hidden_size * 2, hidden_size)

#     def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
#         batch_size = encoder_outputs.size(0)
#         decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)

#         # Reduce bidirectional encoder hidden states to unidirectional decoder hidden states
#         h = torch.cat((encoder_hidden[0][-2], encoder_hidden[0][-1]), dim=1)  # hidden
#         c = torch.cat((encoder_hidden[1][-2], encoder_hidden[1][-1]), dim=1)  # cell
#         decoder_hidden = (self.reduce_h(h).unsqueeze(0), self.reduce_c(c).unsqueeze(0))

#         decoder_outputs = []
#         attentions = []

#         for i in range(MAX_LENGTH):
#             decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, decoder_hidden, encoder_outputs)
#             decoder_outputs.append(decoder_output)
#             attentions.append(attn_weights)

#             if target_tensor is not None:
#                 decoder_input = target_tensor[:, i].unsqueeze(1)  # Teacher forcing
#             else:
#                 _, topi = decoder_output.topk(1)
#                 decoder_input = topi.squeeze(-1).detach()

#         decoder_outputs = torch.cat(decoder_outputs, dim=1)
#         decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
#         attentions = torch.cat(attentions, dim=1)

#         return decoder_outputs, decoder_hidden, attentions

#     def forward_step(self, input_vector, hidden, encoder_outputs):
#         embedded = self.dropout(self.embedding(input_vector))
#         query = hidden[0].permute(1, 0, 2)  # Use hidden state

#         context, attn_weights = self.attention(query, encoder_outputs)
#         input_lstm = torch.cat((embedded, context), dim=2)

#         output, hidden = self.lstm(input_lstm, hidden)
#         output = self.out(output)

#         return output, hidden, attn_weights


In [56]:
# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_vocab_size, dropout_p=0.1):
#         super(AttnDecoderRNN, self).__init__()
#         self.embedding = nn.Embedding(output_vocab_size, hidden_size)
#         self.attention = BahdanauAttention(hidden_size)
#         self.lstm = nn.LSTM(hidden_size * 3, hidden_size, batch_first=True)  # 1x embed + 2x context
#         self.out = nn.Linear(hidden_size, output_vocab_size)
#         self.dropout = nn.Dropout(dropout_p)

#     def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
#         batch_size = encoder_outputs.size(0)
#         decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)

#         # Merge bidirectional LSTM states
#         decoder_hidden = (
#             torch.cat((encoder_hidden[0][-2], encoder_hidden[0][-1]), dim=1).unsqueeze(0),  # hidden state
#             torch.cat((encoder_hidden[1][-2], encoder_hidden[1][-1]), dim=1).unsqueeze(0)   # cell state
#         )

#         decoder_outputs = []
#         attentions = []

#         for i in range(MAX_LENGTH):
#             decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, decoder_hidden, encoder_outputs)
#             decoder_outputs.append(decoder_output)
#             attentions.append(attn_weights)

#             if target_tensor is not None:
#                 decoder_input = target_tensor[:, i].unsqueeze(1)
#             else:
#                 _, topi = decoder_output.topk(1)
#                 decoder_input = topi.squeeze(-1).detach()

#         decoder_outputs = torch.cat(decoder_outputs, dim=1)
#         decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
#         attentions = torch.cat(attentions, dim=1)
#         return decoder_outputs, decoder_hidden, attentions

#     def forward_step(self, input_vector, hidden, encoder_outputs):
#         embedded = self.dropout(self.embedding(input_vector))
#         query = hidden[0].permute(1, 0, 2)  # use hidden state

#         context, attn_weights = self.attention(query, encoder_outputs)
#         input_lstm = torch.cat((embedded, context), dim=2)

#         output, hidden = self.lstm(input_lstm, hidden)
#         output = self.out(output)
#         return output, hidden, attn_weights


In [ ]:
# class BahdanauAttention(nn.Module):
#     def __init__(self, hidden_size):
#         super(BahdanauAttention, self).__init__()
#         self.Wa = nn.Linear(hidden_size, hidden_size)
#         self.Ua = nn.Linear(hidden_size, hidden_size)
#         self.Va = nn.Linear(hidden_size, 1)

#     def forward(self, query, keys):
#         scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
#         scores = scores.squeeze(2).unsqueeze(1)

#         weights = F.softmax(scores, dim=-1)
#         context = torch.bmm(weights, keys)

#         return context, weights

# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, dropout_p=0.1):
#         super(AttnDecoderRNN, self).__init__()
#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.attention = BahdanauAttention(hidden_size)
#         self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.dropout = nn.Dropout(dropout_p)

#     def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
#         batch_size = encoder_outputs.size(0)
#         decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
#         decoder_hidden = encoder_hidden
#         decoder_outputs = []
#         attentions = []

#         for i in range(MAX_LENGTH):
#             decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, decoder_hidden, encoder_outputs)
#             decoder_outputs.append(decoder_output)
#             attentions.append(attn_weights)

#             if target_tensor is not None:
#                 # Teacher forcing: Feed the target as the next input
#                 decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
#             else:
#                 # Without teacher forcing: use its own predictions as the next input
#                 _, topi = decoder_output.topk(1)
#                 decoder_input = topi.squeeze(-1).detach()  # detach from history as input

#         decoder_outputs = torch.cat(decoder_outputs, dim=1)
#         decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
#         attentions = torch.cat(attentions, dim=1)

#         return decoder_outputs, decoder_hidden, attentions


#     def forward_step(self, input, hidden, encoder_outputs):
#         embedded =  self.dropout(self.embedding(input))

#         query = hidden.permute(1, 0, 2)
#         context, attn_weights = self.attention(query, encoder_outputs)
#         input_gru = torch.cat((embedded, context), dim=2)

#         output, hidden = self.gru(input_gru, hidden)
#         output = self.out(output)

#         return output, hidden, attn_weights



# My Model

In [38]:
class MyModel(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size, dropout_p=0.1):
        super(MyModel, self).__init__()
        self.hidden_size = hidden_size

        # Encoder
        self.embedding_enc = nn.Embedding(input_vocab_size, hidden_size)
        self.lstm_enc = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout_enc = nn.Dropout(dropout_p)

        # Attention
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size * 2, hidden_size)  # 2x for bidirectional encoder
        self.Va = nn.Linear(hidden_size, 1)

        # Decoder
        self.embedding_dec = nn.Embedding(output_vocab_size, hidden_size)
        self.lstm_dec = nn.LSTM(hidden_size * 3, hidden_size, batch_first=True)  # embed + 2x context
        self.out = nn.Linear(hidden_size, output_vocab_size)
        self.dropout_dec = nn.Dropout(dropout_p)

        # Projection layers (reduce bi-LSTM encoder outputs)
        self.reduce_h = nn.Linear(hidden_size * 2, hidden_size)
        self.reduce_c = nn.Linear(hidden_size * 2, hidden_size)

    def encode(self, input_vector):
        embedded = self.dropout_enc(self.embedding_enc(input_vector))
        output, (hidden, cell) = self.lstm_enc(embedded)
        return output, (hidden, cell)

    def attention(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)
        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)
        return context, weights

    def forward(self, input_tensor, target_tensor=None):
        # Encode
        encoder_outputs, encoder_hidden = self.encode(input_tensor)
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.full((batch_size, 1), SOS_token, dtype=torch.long, device=input_tensor.device)

        # Reduce bi-LSTM states
        h = torch.cat((encoder_hidden[0][-2], encoder_hidden[0][-1]), dim=1)
        c = torch.cat((encoder_hidden[1][-2], encoder_hidden[1][-1]), dim=1)
        decoder_hidden = (self.reduce_h(h).unsqueeze(0), self.reduce_c(c).unsqueeze(0))

        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            embedded = self.dropout_dec(self.embedding_dec(decoder_input))
            query = decoder_hidden[0].permute(1, 0, 2)

            context, attn_weights = self.attention(query, encoder_outputs)
            input_lstm = torch.cat((embedded, context), dim=2)

            output, decoder_hidden = self.lstm_dec(input_lstm, decoder_hidden)
            output = self.out(output)

            decoder_outputs.append(output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)  # Teacher forcing
            else:
                _, topi = output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


# Prepare training data.

In [39]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('bn', 'en', df)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    print(input_ids.shape)

    # For checking purpose what are the indices associated with sentences
    a = indexesFromSentence(input_lang, pairs[1][0])
    print(a)
    b = indexesFromSentence(output_lang, pairs[1][1])
    print(b)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids       # idx represents pair number.
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    print(input_ids[1])
    print(target_ids[1])

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device), torch.LongTensor(target_ids).to(device))

    print(train_data[1])

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader



In [40]:
input_lang, output_lang, train_dataloader = get_dataloader(32)
for d in train_dataloader:
    a1, b1 = d
    break

print(a1)
print(b1)

Reading lines...
Read 43297 sentence pairs
Counting words...
Counted words:
bn 18920
en 10320
(43297, 27)
[3]
[2]
[3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tensor([3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]), tensor([2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]))
tensor([[  141,   226,  1303,    57,   905,   221,   427,  2626,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [ 2733,  5762,  3145,   285,  3049,  3137,   468, 18486,  7683,  2076,
          6692,  7946,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [ 1903,  1904,  1905,  1410,  1906,   831,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     

# Training the model.

In [41]:
def train_epoch(dataloader, model, optimizer, criterion):
    iterations = 0
    total_loss = 0
    print_loss_total = 0

    for data in dataloader:
        if iterations % 1000 == 0 and iterations != 0:
            print_loss_avg = print_loss_total / 1000.0
            print_loss_total = 0
            print(f"At the {iterations}-th iteration, the training loss is {print_loss_avg}")

        input_tensor, target_tensor = data

        optimizer.zero_grad()
        decoder_outputs, _, _ = model(input_tensor, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print_loss_total += loss.item()
        iterations += 1

    return total_loss / len(dataloader)

In [ ]:
# def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
#     iterations = 0
#     total_loss = 0
#     print_loss_total = 0  # Reset every print_every
    
#     for data in dataloader:
#         if iterations % 1000 == 0:
#             print_loss_avg = print_loss_total / 1000.0
#             print_loss_total = 0
#             print(f"At the {iterations}-th iteration, the training loss is {print_loss_avg}")

#         input_tensor, target_tensor = data

#         encoder_optimizer.zero_grad()       # initialize with gradient zero.
#         decoder_optimizer.zero_grad()

#         encoder_outputs, encoder_hidden = encoder(input_tensor)
#         decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

#         loss = criterion(
#             decoder_outputs.view(-1, decoder_outputs.size(-1)),
#             target_tensor.view(-1)
#         )                                         # calculating loss.
#         loss.backward()

#         encoder_optimizer.step()
#         decoder_optimizer.step()

#         total_loss += loss.item()
#         print_loss_total += loss.item()
#         iterations += 1

#     return total_loss / len(dataloader)

In [42]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [43]:
def train(train_dataloader, model, n_epochs, learning_rate=0.001):
    start = time.time()
    losses_list = []

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, model, optimizer, criterion)
        losses_list.append(loss)
        print(f"At the {epoch}-th epoch :-")
        print('%s %.4f' % (timeSince(start, epoch / n_epochs), loss))

    return losses_list


In [ ]:
# def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001):
#     start = time.time()
#     losses_list = []

#     encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
#     criterion = nn.NLLLoss()

#     for epoch in range(1, n_epochs + 1):
#         loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
#         losses_list.append(loss)
#         print(f"At the {epoch}-th epoch :-")
#         print('%s %.4f' % (timeSince(start, epoch / n_epochs), loss))

#     return losses_list
    

# Training.

In [44]:
hidden_size = 128
batch_size = 32
n_epochs = 10
learning_rate = 0.001

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

model = MyModel(input_lang.n_words, output_lang.n_words, hidden_size).to(device)

Reading lines...
Read 43297 sentence pairs
Counting words...
Counted words:
bn 18920
en 10320
(43297, 27)
[3]
[2]
[3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tensor([3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]), tensor([2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]))


In [45]:
model.train()
loss_list = train(train_dataloader, model, n_epochs, learning_rate)

At the 1000-th iteration, the training loss is 2.0399986044168474
At the 1-th epoch :-
14m 58s (- 134m 42s) 1.9294


KeyboardInterrupt: 

In [ ]:
# hidden_size = 128
# batch_size = 32
# n_epochs = 10
# learning_rate=0.001

# input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

# encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)


Reading lines...
Read 43297 sentence pairs
Counting words...
Counted words:
bn 18920
en 10320
(43297, 27)
[3]
[2]
[3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tensor([3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0'), tensor([2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0'))


In [46]:
model.parameters

<bound method Module.parameters of MyModel(
  (embedding_enc): Embedding(18920, 128)
  (lstm_enc): LSTM(128, 128, batch_first=True, bidirectional=True)
  (dropout_enc): Dropout(p=0.1, inplace=False)
  (Wa): Linear(in_features=128, out_features=128, bias=True)
  (Ua): Linear(in_features=256, out_features=128, bias=True)
  (Va): Linear(in_features=128, out_features=1, bias=True)
  (embedding_dec): Embedding(10320, 128)
  (lstm_dec): LSTM(384, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=10320, bias=True)
  (dropout_dec): Dropout(p=0.1, inplace=False)
  (reduce_h): Linear(in_features=256, out_features=128, bias=True)
  (reduce_c): Linear(in_features=256, out_features=128, bias=True)
)>

In [ ]:
# encoder.parameters

<bound method Module.parameters of EncoderRNN(
  (embedding): Embedding(18920, 128)
  (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
)>

In [ ]:
# decoder.parameters

<bound method Module.parameters of AttnDecoderRNN(
  (embedding): Embedding(10320, 128)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=128, out_features=128, bias=True)
    (Ua): Linear(in_features=256, out_features=128, bias=True)
    (Va): Linear(in_features=128, out_features=1, bias=True)
  )
  (lstm): LSTM(384, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=10320, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (reduce_h): Linear(in_features=256, out_features=128, bias=True)
  (reduce_c): Linear(in_features=256, out_features=128, bias=True)
)>

In [ ]:
# encoder.train()
# decoder.train()
# loss_list = train(train_dataloader, encoder, decoder, n_epochs, learning_rate)

At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.2532339130192995
At the 1-th epoch :-
1m 21s (- 12m 16s) 0.2570
At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.2474772524535656
At the 2-th epoch :-
2m 43s (- 10m 54s) 0.2513
At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.24472207069396973
At the 3-th epoch :-
4m 5s (- 9m 33s) 0.2488
At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.2445441931784153
At the 4-th epoch :-
5m 27s (- 8m 10s) 0.2479
At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.24096273820102215
At the 5-th epoch :-
6m 49s (- 6m 49s) 0.2442
At the 0-th iteration, the training loss is 0.0
At the 1000-th iteration, the training loss is 0.2378548332452774
At the 6-th epoch :-
8m 10s (- 5m 27s) 0.2420
At the 0-th iteration, the training l

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_list)), loss_list, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [133]:
loss_list

[0.2843785828290345,
 0.2790027569808847,
 0.27457336275845196,
 0.2715281860316874,
 0.2683845982890978,
 0.26555534293448646,
 0.2627923678788336,
 0.2607242783182129,
 0.2581127020289662,
 0.25504225606932424]

# Evaluation.

In [47]:
def evaluate(model, sentence, input_lang, output_lang):
    model.eval()
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)       # creating tensor from sentence.

        #encoder_outputs, encoder_hidden = encoder(input_tensor)
        #decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)
        decoder_outputs, decoder_hidden, decoder_attn = model(input_tensor)

        _, topi = decoder_outputs.topk(1)           # searching the index with high probabilities.
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [56]:
def evaluateRandomly(model, pairs, input_lang, output_lang, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(model, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [49]:
pairs[:5]

[['যান', 'go'],
 ['যা', 'go'],
 ['পালাও', 'run'],
 ['পালান', 'run'],
 ['কে', 'who']]

In [50]:
input_lang

In [51]:
output_lang

In [52]:
dummy_pair = [
    ['আমি খুব ভালো', 'I am very good'],
    ['দরজা খোল', 'Open the door'],
    ['সে খুব খুশি', 'He is very happy'],
    ['যান', 'go']
]


In [53]:
def evaluateBLEU(model, pairs, input_lang, output_lang, n=100):
    smoothie = SmoothingFunction().method4
    total_score = 0

    for i in range(n):
        pair = random.choice(pairs)
        output_words, _ = evaluate(model, pair[0], input_lang, output_lang)

        # Remove <EOS> and other special tokens from predicted output
        output_words = [word for word in output_words if word not in ['<EOS>', '<PAD>', '<SOS>']]

        # Prepare reference (true translation) token list
        reference = pair[1].split()  # ground truth
        candidate = output_words     # model output

        # Calculate BLEU score for the current pair
        score = sentence_bleu([reference], candidate, smoothing_function=smoothie)
        total_score += score

        print(f"Input: {pair[0]}")
        print(f"Target: {pair[1]}")
        print(f"Predicted: {' '.join(candidate)}")
        print(f"BLEU: {score:.4f}\n")

    avg_bleu = total_score / n
    print(f"Average BLEU score over {n} samples: {avg_bleu:.4f}")
    return avg_bleu


In [54]:
evaluateBLEU(model, pairs, input_lang, output_lang, n=50)

Input: মুখে লাঠি নিয়ে একটি কুকুর সাঁতার কাটছ
Target: a dog swimming though water holding a stick
Predicted: a dog with a dog is running through the grass
BLEU: 0.0630

Input: দর্শকেরা একটি জার্মান শেফার্ড কুকুরের বিভিন্ন কসরত উপভোগ করছে
Target: onlookers watch a german shephard climb steps on an obstacle course
Predicted: a dog with a dog
BLEU: 0.0137

Input: লাল জামা-কালো প্যান্ট পরিহিত একটি লোক ও সাদা জামা-তামাটে ছোটো প্যান্ট পরিহিত একটি লোক গল্ফ খেলছ
Target: one man in red shirt and black pants with a man in white shirt and tan shorts , golfing
Predicted: a man in a red shirt and white shirt and a red shirt and a red shirt is playing with a red shirt
BLEU: 0.1327

Input: দুইটি কুকুর বরফের মধ্যে লড়াই করছ
Target: two dogs are fighting in the snow
Predicted: two dogs are running through the grass
BLEU: 0.2075

Input: চারজন ব্যক্তি শহুরে ফুটপাথে হাটছেন এবং সবুজ টুপি পরা ব্যক্তি সেলফোনে কথা বলছে
Target: four people are walking along a city sidewalk , while the one in the green hat talks

0.062342841055212365

In [57]:
evaluateRandomly(model, dummy_pair, input_lang, output_lang, n = 3)

> আমি খুব ভালো
= I am very good
< i am SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> দরজা খোল
= Open the door
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS

> যান
= go
< SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS



# Visualization

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(model, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])




In [ ]:
evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive <EOS>
input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>


<ipython-input-22-e5a954b36ffc>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-22-e5a954b36ffc>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
